In [21]:
from numpy import *
from os import listdir 
import operator

### Example: a handwriting recognition system

Now let's see how to apply kNN to things as diverse as images where the data is in binary form. 

We’ll be working only with the digits 0–9. These digits were processed through image-processing software to make them all the same size and color. The dataset is a modified version of the “Optical Recognition of Handwritten Digits Data Set” by E. Alpaydin, C. Kaynak, Department of Computer Engineering at Bogazici University, 80815 Istanbul Turkey, retrieved from the UCI Machine Learning Repository (http://archive.ics.uci.edu/ml) on October 3, 2010.

They’re all 32x32 black and white. The binary images were converted to text format to make this example easier, although it isn’t the most efficient use of memory.

> Example: using kNN on a handwriting recognition system
 > 1. Collect: Text file provided.
 > 2. Prepare: Write a function to convert from the image format to the list format  used in our classifier, classifyNum().
 > 3. Analyze: We’ll look at the prepared data in the Python shell to make sure it’s correct. 
 > 4. Train: Doesn’t apply to the kNN algorithm.
 > 5. Test: Write a function to use some portion of the data as test examples. The test examples are classified against the non-test examples. If the predicted class doesn’t match the real class, we’ll count that as an error.
 > 6. Use: Not performed in this example. We could build a complete program to extract digits from an image, such a system used to sort the mail in the United States

##### Step 1: Prepare: converting images into test vectors

We’ll use the trainingDigits directory to train our classifier and testDigits to test it. There’ll be no overlap between the two groups.

We’d like to use the same classifier that we used in the previous two examples, so we’re going to need to reformat the images to a single vector. 

We’ll take the 32x32 matrix that is each binary image and make it a 1x1024 vector. After we do this, we can apply it to our existing classifier. 

The following code is a small function called `img2vector`, which converts the image to a vector. The function creates a 1x1024 NumPy array, then opens the given file, loops over the first 32 lines in the file, and stores the integer value of the first 32 characters on each line in the NumPy array. This array is finally returned.

In [1]:
def img2vector(filename):
    returnVect = zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0, 32*i+j] = int(lineStr[j])
    return returnVect

In [6]:
testVector = img2vector('Machine_Learning_In_Action/testDigits/0_13.txt')

testVector[0,0:31]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

##### Step 2: Test: kNN on handwritten digits

In [8]:
# import the same classifyKNN() function from the first notebook
def classifyKNN(inX, dataSet, labels, k):
    dataSetSize = dataSet.shape[0]
    diffMat = tile(inX, (dataSetSize, 1)) - dataSet
    sqDiffMat = diffMat ** 2
    sqDistances = sqDiffMat.sum(axis = 1)
    distances = sqDistances ** 0.5
    sortedDisIndicies = distances.argsort()
    classCount = {}
    for i in range(k):
        voteIlabel = labels[sortedDisIndicies[i]]
        classCount[voteIlabel] = classCount.get(voteIlabel, 0) + 1
    sortedClassCount = sorted(classCount.items(), key = operator.itemgetter(1), reverse = True)
    return sortedClassCount[0][0]

In [27]:
def handwritingClassTest():
    # get contents from the training directory
    hwLabels = []
    trainingFileList = listdir('Machine_Learning_In_Action/trainingDigits')   
    m = len(trainingFileList)
    trainingMat = zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]        
        fileStr = fileNameStr.split('.')[0]             
        classNumStr = int(fileStr.split('_')[0]) 
        hwLabels.append(classNumStr)
        trainingMat[i,:] = img2vector('Machine_Learning_In_Action/trainingDigits/%s' % fileNameStr)
    
    # get contents from the test directory
    testFileList = listdir('Machine_Learning_In_Action/testDigits')
    errorCount = 0.0
    mTest = len(testFileList)
    errorFileLocation = []
    for i in range(mTest):
        fileNameStr = testFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        vectorUnderTest = img2vector('Machine_Learning_In_Action/testDigits/%s' % fileNameStr)
        classifierResult = classifyKNN(vectorUnderTest, trainingMat, hwLabels, 3)
        # Personally I don't want the following print command, since I don't want to see all details
#         print("the classifier came back with: %d, the real answer is: %d" % (classifierResult, classNumStr))
        # so here I only show this message while the prediction is wrong
        if (classifierResult != classNumStr): 
            errorCount += 1.0
            errorFileLocation.append(i)
            print("the classifier came back with: %d, the real answer is: %d" % (classifierResult, classNumStr))
    print("\nthe total number of errors is: %d" % errorCount)
    print("\nthe total error rate is: %f" % (errorCount/float(mTest)))
    return errorFileLocation

In [14]:
# Now let's dive into the function before running it as a whole
# see how many training and test images
trainingFileList = listdir('Machine_Learning_In_Action/trainingDigits')  

In [15]:
len(trainingFileList)

1934

In [16]:
testFileList = listdir('Machine_Learning_In_Action/testDigits')

In [17]:
len(testFileList)

946

In [18]:
trainingFileList[:3]
# The filename is something like 9_45.txt, where 9 is the class number and it is the 45th instance of the digit 9. 

['0_0.txt', '0_1.txt', '0_10.txt']

In [28]:
errorFileLocation = handwritingClassTest()

the classifier came back with: 7, the real answer is: 1
the classifier came back with: 9, the real answer is: 3
the classifier came back with: 3, the real answer is: 5
the classifier came back with: 6, the real answer is: 5
the classifier came back with: 6, the real answer is: 8
the classifier came back with: 3, the real answer is: 8
the classifier came back with: 1, the real answer is: 8
the classifier came back with: 1, the real answer is: 8
the classifier came back with: 1, the real answer is: 9
the classifier came back with: 7, the real answer is: 9

the total number of errors is: 10

the total error rate is: 0.010571


In [25]:
errorFileLocation

[172, 279, 520, 521, 769, 782, 796, 806, 863, 914]

In [26]:
testFileList[172]

'1_86.txt'

Using the kNN algorithm on this dataset, we were able to achieve an error rate of 1.05%.

### Next Steps

We can 
- vary k to see how this changes
- modify the handwritingClassTest() function to randomly select training examples, i.e., vary the number of training examples and see how that impacts the error rate
- apply kD-trees, a modification of kNN, which reduces the number of calculations.

### Summary

The k-Nearest Neighbors algorithm is a simple and effective way to classify data. The examples in this chapter should be evidence of how powerful a classifier it is. 

kNN is an example of instance-based learning, where we need to have instances of data close at hand to perform the machine learning algorithm. The algorithm has to carry around the full dataset; for large datasets, this implies a large amount of storage. In addition, we need to calculate the distance measurement for every piece of data in the database, and this can be cumbersome. 

An additional drawback is that kNN doesn’t give us any idea of the underlying structure of the data; we have no idea what an *average* or *exemplar* instance from each class looks like. 

In the next chapter, we’ll address this issue by exploring ways in which probability measurements can help us do classification.